This notebook will achieve the following:
1.  Download the breast cancer segmentation dataset
2.  Remap the ground truth segmentation masks from 22 classes to 6 classes
3. Split the whole slide images and masks into sub-sections (patches) 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
#system tools
from functools import partial
from math import floor,ceil
import multiprocessing
import os
import shutil
import typing
from typing import Callable, List
import time

#vis tools
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

#image processing
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from PIL import Image

#other
import pandas as pd

## Getting the cancer dataset:

We download the breast cancer segmentation dataset as detailed in:
https://github.com/PathologyDataScience/BCSS



In [ ]:
class PrepareDataset():

  def __init__(self):
    #
    palette = sns.color_palette('tab20', 6) #make new palette                                            
    palette_array = np.asarray(palette)*255
    palette_array = np.roll(palette_array, -2,axis=0) # roll array so orange is tumour 
    self.palette_array = palette_array

  def clone_repo(self, 
                 path_to_clone_repo:str):
    #clone the repo so we can download the images
    self.path_to_clone_repo = os.path.abspath(path_to_clone_repo)
    if not os.path.exists(path_to_clone_repo):
      !git clone https://github.com/CancerDataScience/CrowdsourcingDataset-Amgadetal2019 $path_to_clone_repo
      !cd $path_to_clone_repo

  def download_dataset(self, 
                       path_to_download_dataset:str):
    

    #check if dataset has already been downloaded if not download it
    if os.path.exists(path_to_download_dataset):
      print(f'it appears the dataset has already been downloaded at {path_to_download_dataset}')
    else:
      os.chdir(self.path_to_clone_repo)
      !pip install girder_client girder-client pillow numpy scikit-image imageio #install needed packages to run the download script
      
      #modify the config file to point to the path where we want to download the images instead of default save path
      path_to_config_file = os.path.join(os.getcwd(),'configs.py')
      path_to_new_config_file = os.path.join(os.getcwd(),'configs_new.py')
      
      #read previous config file so we can rewrite/edit the contents
      old_config_lines = []
      with open(path_to_config_file,'r') as conf_file:
        old_config_lines = conf_file.readlines()

      with open(path_to_new_config_file,'w') as conf_file:
        for line in old_config_lines:
          new_line = line.replace("SAVEPATH = './'",f"SAVEPATH = '{path_to_download_dataset}'")
          conf_file.write(new_line+'\n')

      #keep old config file incase we need it for reference
      os.rename(path_to_config_file,'/content/gdrive/MyDrive/BCSS_segmentation_repo/configs_old.py')
      os.rename(path_to_new_config_file,'/content/gdrive/MyDrive/BCSS_segmentation_repo/configs.py')
      print('new config file created')

      #download the images
      print('starting download of dataset:')
      !python download_crowdsource_dataset.py
      print('finished download of dataset:')

    self.orig_dataset_path = path_to_download_dataset
    os.chdir('/content')
    orig_img_folder_name = os.path.join(path_to_download_dataset,'images')
    if os.path.exists(orig_img_folder_name):
      print('renaming original image folder')
      new_img_folder_name = os.path.join(path_to_download_dataset,'orig_images')
      os.rename(orig_img_folder_name,new_img_folder_name)
 
  #make lists of filepaths for original images and masks
  def get_orig_file_paths(self):
    '''
    get filepaths for original images and segmentation masks and save them as two lists
    '''
    self.orig_images_folder_path = os.path.join(self.orig_dataset_path,'orig_images')
    self.orig_masks_folder_path = os.path.join(self.orig_dataset_path,'masks')

    #make list of original image paths
    orig_image_paths = []
    for img in os.listdir(self.orig_images_folder_path):
      temp_img_path = os.path.join(self.orig_images_folder_path,img)
      orig_image_paths.append(temp_img_path)
    self.orig_image_paths_list = sorted(orig_image_paths)

    #make list of original mask paths
    orig_mask_paths = []
    for mask in os.listdir(self.orig_masks_folder_path):
      temp_mask_path = os.path.join(self.orig_masks_folder_path,mask)
      orig_mask_paths.append(temp_mask_path)
      self.orig_mask_paths_list = sorted(orig_mask_paths)

  #define  function to remap ground truth pixel values as defined in original paper
  def map_gt_to_new_values_6_class(self,
                                   orig_gt:int)-> int:

    if orig_gt in [1,19,20]: # if gt is tumor, angioinvasion, dcis
      return 1 #return tumor class
    elif orig_gt in [2]: # if gt is stroma
      return 2 #return stroma class
    elif orig_gt in [3,10,11]: #if gt is lymphocytes , plasma cells, other immune infiltrates
      return 3 #return inflammatory class
    elif orig_gt in [4]: #if gt is necrosis_or_debris
      return 4 #return necrosis class
    elif orig_gt in [0]: #if gt is outside ROI
      return 0 #return outside ROI
    else: 
      return 5 #return other class

  def map_gt_to_new_values_2_class(self,
                                   orig_gt:int) -> int:

    if orig_gt in [1,19,20]: # if gt is tumor, angioinvasion, dcis
      return 1 #return tumor class
    else: 
      return 0 #return other class
  
  
  def process_mask(self,
                   mask_path:str, 
                   new_masks_folder:str,
                   vfunc_to_convert:Callable,
                   make_3d_mask:bool=False) -> str:
    '''
    converts ground truth values for a single mask
    '''
    img_name = os.path.basename(mask_path)
    print(f'processing mask: {img_name}')
    mask_img = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)

    # Map pixel values of ground truth masks using mapping to 6 classes or binary (tumor=0,1)
    if vfunc_to_convert == None:
      vfunc_to_convert = np.vectorize(self.map_gt_to_new_values_6_class)
    new_mask = vfunc_to_convert(mask_img) #apply conversion function to remap pixel values
    
    #if 3d mask is required, copy 2d image across RGB channels
    if make_3d_mask==True:
      new_mask = np.dstack((new_mask,new_mask,new_mask))
    
    if not os.path.exists(new_masks_folder):
      os.makedirs(new_masks_folder)
    temp_path = os.path.join(new_masks_folder, img_name)
    print(f'temp path: {temp_path}')
    cv2.imwrite(temp_path, new_mask)
    print(f'finished mask: {img_name}')
    return temp_path

  def remap_ground_truth_masks(self,
                               new_masks_folder:str,
                               convert_to_binary:bool,
                               make_3d_mask:bool):
    '''
    remap pixel values of seg mask

    Original ground truth includes ~20 classes see /0_Public-data-Amgad2019_0.25MPP/meta/gtruth_codes.tsv
    We want to collapse this to 6 classes of: 'outside_roi','tumor','stroma','inflammatory','necrosis','other'
    to match with the paper. 

    inputs:
    new_masks_folder [str] = folder to save the adjusted ground truth masks
    remap_to [str] = '2d' or '3d' , whether to remap the mask to a 2d or 3d image
    '''

    self.new_masks_folder = new_masks_folder
    new_masks_paths = []

    #if the folder for the modified ground truth masks does not exist, create it
    if os.path.exists(new_masks_folder):
      user_choice = input('new masks folder already exists, confirm replace existing new masks folder (y/n)')
      if user_choice == 'y':
        shutil.rmtree(new_masks_folder)
      else:
        for mask in os.listdir(new_masks_folder):
          temp_path = os.path.join(os.path.dirname(new_masks_folder),new_masks_folder,mask)
          new_masks_paths.append(temp_path)
        self.new_masks_paths_list = new_masks_paths
        print('nothing to do, exiting')
        return None
      
    os.makedirs(new_masks_folder)
    if convert_to_binary == False:
      vfunc = np.vectorize(self.map_gt_to_new_values_6_class)
    else:
      vfunc = np.vectorize(self.map_gt_to_new_values_2_class)
    
    x = 0
    total_masks = len(self.orig_mask_paths_list)
    print('creating new image masks')

    for mask_path in self.orig_mask_paths_list:
      print(f'mask {x}/total_masks')
      temp_path = self.process_mask(mask_path=mask_path, 
                        new_masks_folder=new_masks_folder,
                        vfunc_to_convert=vfunc,
                        make_3d_mask=make_3d_mask)
      new_masks_paths.append(temp_path)
      x+=1
    # self.new_masks_paths_list = self.process_masks_parallel(new_masks_folder=new_masks_folder,
    #                                                remap_to=remap_to)
    self.new_masks_paths_list = new_masks_paths                                        
    print('finished creating new image masks')
    
  
  def sanity_check(self,image_list:List,mask_list:List):
    '''
    check list of images and masks has the same length and that the filenames are matching
    '''

    try:
      assert (len(image_list)==len(mask_list))
      print('image and mask lists are the same length')
    except:
      print(f'len image list : {image_list} , len mask list : {mask_list}')
      raise ValueError('list of image paths and list of mask paths are not the same length')

    print('for each x image and mask should have same name')
    for x in range(5):
      print(x)
      print('orig image')
      print(f'{image_list[x]}')
      print('mask')
      print(f'{mask_list[x]}')

  #get bounding boxes for image patches
  def calculate_slice_bboxes(self,
                             input_img_path:str,
                             slice_size:int=800)->List[List[int]]:

    """
    calculates how to divide an image into overlapping slices according to patch size. 
    
    calculates how an image should be divided into overlapping patches and
    returns a list of bounding boxes where each bounding box 

    inputs:
    input_img: str 
      path to image to be sliced
    slice_size: int
      patch size - cuts image into patches of dims slice_size x slice_size
    
    returns:
    slice_bboxes: list of lists:
      a list of n lists where each inner list corresponds to a single patch in the format x1y1x2y2
    """

    #load image
    test_img = cv2.imread(input_img_path)
    test_img_np = np.asarray(test_img)
    im_dims = np.shape(test_img_np)
    #get height and width
    image_width = im_dims[1]
    image_height = im_dims[0]

    #calculate variables for making patches
    wid_over_p_size = image_width/slice_size #total width/patch size : gives n of patches in decimal to cover whole image
    n_width_patches_to_get_whole_width = ceil(image_width/slice_size) #can't have decimal patches so round up number of patches needed
    width_overlap_total = n_width_patches_to_get_whole_width*slice_size - image_width #if we make patches with no overlap we overshoot the edge of the image by a certain number of pixels
    width_overlap_between_two_patches = width_overlap_total/(n_width_patches_to_get_whole_width-1) 

    height_over_p_size = image_height/slice_size #total width/patch size : gives n of patches in decimal to cover whole image
    n_height_patches_to_get_whole_height= ceil(image_height/slice_size) #can't have decimal patches so round up number of patches needed
    height_overlap_total = n_height_patches_to_get_whole_height*slice_size - image_height #if we make patches with no overlap we overshoot the edge of the image by a certain number of pixels
    height_overlap_between_two_patches = height_overlap_total/(n_height_patches_to_get_whole_height-1) 
    
    #init lists
    x_mins = []
    x_maxs = []
    y_mins = []
    y_maxs = []
    
    #find patch limits as patch moves across the x direction from left to right:
    for x in range(n_width_patches_to_get_whole_width):
      if x == 0:
        x_min = 0
        x_max = slice_size
      else:
        x_min = int(x_min+slice_size - width_overlap_between_two_patches)
        x_max = int(x_min+slice_size)
        if x_max > image_width:
          x_max = image_width
          x_min = image_width - slice_size
      x_mins.append(x_min)
      x_maxs.append(x_max)

    #find patch limits as patch moves up the y direction from bottom to top:
    for y in range(n_height_patches_to_get_whole_height):
      if y == 0:
        y_min = 0
        y_max = slice_size
      else:
        y_min = int(y_min+slice_size - height_overlap_between_two_patches)
        y_max = int(y_min+slice_size)
        if y_max > image_height:
          y_max = image_height
          y_min = image_height - slice_size
      y_mins.append(y_min)
      y_maxs.append(y_max)

    # convert lists to bbox xyxy
    slice_bboxes = []
    for y in range(len(y_mins)):
      for x in range(len(x_mins)):
        x1 = x_mins[x]
        y1 = y_mins[y]
        x2 = x_maxs[x]
        y2 = y_maxs[y]
        slice_bboxes.append([x1,y1,x2,y2])

    return slice_bboxes

  #split a single image into patches
  def split_and_save_single_image_and_mask(self,
                                           orig_image_path:str,
                                           mask_path:str,
                                           output_folder:str=None,
                                           slice_size:int=800):
    '''split an input image and associated mask into 800x800 sections

    The original images and masks are at least ~2000 pixels in height and width
    In the original paper, they use image section of 800x800 and so we modify the original images and masks
    and split them into sections of 800x800 images

    inputs:
    image_name: path to a single image
    output_folder: a folder to save the 800x800 image sections 
    image_type: choose 'img' or 'mask'. Only difference is for image we need to specify the third image dimension
    '''
    new_image_folder = os.path.join(output_folder,'images')
    new_mask_folder = os.path.join(output_folder,'masks')
    if not os.path.exists(new_image_folder):
      os.makedirs(new_image_folder)
    if not os.path.exists(new_mask_folder):
      os.makedirs(new_mask_folder)

    img_array = np.asarray(cv2.imread(orig_image_path))
    img_name = orig_image_path.split('/')[-1].split('.')[0] #get image name from path
    mask_array = np.asarray(cv2.imread(mask_path,cv2.IMREAD_UNCHANGED)) #read image in same format 

    
    # print(f'shape of img array: {np.shape(img_array)}')
    # print(f'shape of mask array: {np.shape(mask_array)}')

    #get bboxes for input image
    bboxes_sing_img_patches = self.calculate_slice_bboxes(orig_image_path,slice_size)
    print(f'splitting {img_name} into {len(bboxes_sing_img_patches)} patches:')

    # start_time = time.perf_counter()
    # loop over the image patches and save each patch as a new image
    for x in range(len(bboxes_sing_img_patches)):
      temp_bbox = bboxes_sing_img_patches[x] #xyxy
      temp_x_start = temp_bbox[0]
      temp_x_end = temp_bbox[2]
      temp_y_start = temp_bbox[1]
      temp_y_end = temp_bbox[3]

      new_name = f"{img_name}_sub_image_{x:02d}.png" #create new image name,with number format 01 , 02... 10,11 etc

      #create image patches and save
      temp_img_patch = img_array[temp_y_start:temp_y_end,temp_x_start:temp_x_end,:] 
      temp_mask_patch = mask_array[temp_y_start:temp_y_end,temp_x_start:temp_x_end]
      
      #create paths for newly created files
      new_image_path = os.path.join(new_image_folder,new_name) #make path to save image section
      new_mask_path = os.path.join(new_mask_folder,new_name)
      
      #save single patch for image and mask
      cv2.imwrite(new_image_path,temp_img_patch)
      cv2.imwrite(new_mask_path,temp_mask_patch)

    print(f'finished splitting {img_name}')

    # end_time = time.perf_counter()
    # execution_time = end_time - start_time
    # print(f"The execution time to patch an image with cv2 is: {execution_time}")

  def start_splitting_all_images_masks(self, new_folder_for_patched:str, 
                                       mask_type:str='new'):
    '''
    start splitting all images and masks
    '''

    self.patched_images_folder = os.path.join(new_folder_for_patched,'images')
    self.patched_masks_folder = os.path.join(new_folder_for_patched,'masks')

    patched_images_paths_list = []
    patched_masks_paths_list = []

    if not os.path.exists(new_folder_for_patched):
      os.makedirs(new_folder_for_patched)
    else:
      user_input = input('it appears patch folder already exists. confirm delete existing folder (y/n)')
      if user_input == 'y':
        print('deleting existing patch folder')
        shutil.rmtree(new_folder_for_patched)
        os.makedirs(self.patched_images_folder)
        os.makedirs(self.patched_masks_folder)
      else: #get existing paths for patched images and masks
        for patched_img in os.listdir(self.patched_images_folder):
          temp_path = os.path.join(os.path.dirname(self.patched_images_folder),patched_img)
          patched_images_paths_list.append(temp_path)
        self.patched_images_paths_list = patched_images_paths_list

        for patched_mask in os.listdir(self.patched_masks_folder):
          temp_path = os.path.join(os.path.dirname(self.patched_masks_folder),patched_mask)
          patched_masks_paths_list.append(temp_path)
        self.patched_masks_paths_list = patched_masks_paths_list
        print('nothing to do, exiting')
        return None

    # get list of image and mask pairs to process
    image_mask_pairs = zip(self.orig_image_paths_list, self.new_masks_paths_list if mask_type == 'new' else self.orig_masks_paths_list)
    self.split_all_input_images_and_masks_opt(image_mask_pairs=image_mask_pairs,
                                         new_folder_for_patched=new_folder_for_patched)

  def split_all_input_images_and_masks_opt(self, image_mask_pairs:zip,new_folder_for_patched:str):
    '''
    use multiprocessing for efficient pacthing of images and corresponding masks
    '''
    
    # create a pool of worker processes
    pool = multiprocessing.Pool()

    # apply the process_image function to each image and mask pair in the input folder
    results = []
    for image_path, mask_path in image_mask_pairs:
        result = pool.apply_async(self.split_and_save_single_image_and_mask, args=(image_path, mask_path, new_folder_for_patched, 800))
        results.append(result)

    # wait for all processes to complete and get results
    for result in results:
        result.wait()
        # print(result.get())

    print('all images and masks split into patches')

  def get_patched_file_paths(self,patched_folder_path:str):
    '''
    get filepaths for patched images and segmentation masks and save them as two lists

    inputs: patched_folder_path
    '''
    self.patched_images_folder_path = os.path.join(patched_folder_path,'images')
    self.patched_masks_folder_path = os.path.join(patched_folder_path,'masks')

    #make list of original image paths
    patched_image_paths = []
    for img in os.listdir(self.patched_images_folder_path):
      temp_img_path = os.path.join(self.patched_images_folder_path,img)
      patched_image_paths.append(temp_img_path)
    self.patched_image_paths_list = sorted(patched_image_paths)

    #make list of original mask paths
    patched_mask_paths = []
    for mask in os.listdir(self.patched_masks_folder_path):
      temp_mask_path = os.path.join(self.patched_masks_folder_path,mask)
      patched_mask_paths.append(temp_mask_path)
    self.patched_mask_paths_list = sorted(patched_mask_paths)

  def make_sure_same_images_and_masks_present(self):

    image_patch_names = set([path.split('/')[-1] for path in self.patched_image_paths_list])
    mask_patch_names = set([path.split('/')[-1] for path in self.patched_mask_paths_list])

    # Identify missing image names from mask list
    missing_image_names_from_mask_list = list(image_patch_names - mask_patch_names)

    # Identify missing mask names from image list
    missing_mask_names_from_image_list = list(mask_patch_names - image_patch_names)

    print(f'num images not present in mask list: {len(missing_image_names_from_mask_list)}')
    print(f'num masks not present in image list: {len(missing_mask_names_from_image_list)}')

    bad_image_names = []
    bad_mask_names = []
    image_patch_names_list = list(image_patch_names)
    mask_patch_names_list = list(mask_patch_names)

    for name in image_patch_names_list:
      if ('.png.png' in name):
        print(f'bad filename: {name}')
        bad_image_names.append(name)

    for name in mask_patch_names_list:
        if ('.png.png' in name):
          print(f'bad filename: {name}')
          bad_mask_names.append(name)

    print(f'total paths with bad filename ".png.png": {len(bad_image_names)+len(bad_mask_names)}')

    user_choice = input('confirm delete unmatching images and masks (y/n)')
    if user_choice == 'y':
    # Remove missing filenames from each list
      for name in missing_image_names_from_mask_list+bad_image_names:
          path_to_remove = os.path.join(self.patched_images_folder_path,name)
          self.patched_image_paths_list.remove(path_to_remove)

      for name in missing_mask_names_from_image_list+bad_mask_names:
          path_to_remove = os.path.join(self.patched_masks_folder_path,name)
          self.patched_mask_paths_list.remove(path_to_remove)

      print('images and masks now all have mnatching pairs')
    else:
      print('nothing to do. \n Warning: error will be thrown during training if images/masks are not both present in respective folders')

prep_dataset_obj = PrepareDataset()

In [ ]:
#download dataset
prep_dataset_obj.clone_repo(path_to_clone_repo='/content/gdrive/MyDrive/BCSS_segmentation_repo')
prep_dataset_obj.download_dataset(path_to_download_dataset='/content/gdrive/MyDrive/BCSS_dataset')

it appears the dataset has already been downloaded at /content/gdrive/MyDrive/BCSS_dataset


##Setting paths to images/masks

In [ ]:
prep_dataset_obj.get_orig_file_paths()
prep_dataset_obj.sanity_check(prep_dataset_obj.orig_image_paths_list,prep_dataset_obj.orig_mask_paths_list)

image and mask lists are the same length
for each x image and mask should have same name
0
orig image
/content/gdrive/MyDrive/BCSS_dataset/orig_images/TCGA-A1-A0SK-DX1_xmin45749_ymin25055_MPP-0.2500.png
mask
/content/gdrive/MyDrive/BCSS_dataset/masks/TCGA-A1-A0SK-DX1_xmin45749_ymin25055_MPP-0.2500.png
1
orig image
/content/gdrive/MyDrive/BCSS_dataset/orig_images/TCGA-A1-A0SP-DX1_xmin6798_ymin53719_MPP-0.2500.png
mask
/content/gdrive/MyDrive/BCSS_dataset/masks/TCGA-A1-A0SP-DX1_xmin6798_ymin53719_MPP-0.2500.png
2
orig image
/content/gdrive/MyDrive/BCSS_dataset/orig_images/TCGA-A2-A04P-DX1_xmin104246_ymin48517_MPP-0.2500.png
mask
/content/gdrive/MyDrive/BCSS_dataset/masks/TCGA-A2-A04P-DX1_xmin104246_ymin48517_MPP-0.2500.png
3
orig image
/content/gdrive/MyDrive/BCSS_dataset/orig_images/TCGA-A2-A04Q-DX1_xmin20868_ymin17804_MPP-0.2500.png
mask
/content/gdrive/MyDrive/BCSS_dataset/masks/TCGA-A2-A04Q-DX1_xmin20868_ymin17804_MPP-0.2500.png
4
orig image
/content/gdrive/MyDrive/BCSS_dataset/orig_

### Remapping ground truth classes:

https://bcsegmentation.grand-challenge.org/Baseline/

Class grouping
The network was trained to map pixels into five region classes: 
**tumor, stroma, inflammatory infiltration, necrosis** and other. Regions that belong to rare classes were grouped with predominant classes where appropriate, as follows:

Grouped with “tumor”: angioinvasion, DCIS.
Grouped with “inflammatory infiltrates”: lymphocytes, plasma cells, other immune infiltrates. 



#### Pandas tables of old and new classes for reference:


In [ ]:
ground_truth_tsv_path =os.path.join(prep_dataset_obj.path_to_clone_repo,'meta/gtruth_codes.tsv')

cancer_classes = pd.read_csv(ground_truth_tsv_path,
                             sep = '\t')
 
cancer_classes['new'] = cancer_classes['GT_code'].map(prep_dataset_obj.map_gt_to_new_values_6_class)
orig_gt_labels = cancer_classes.GT_code.values
cancer_classes

,label,GT_code,new
0,outside_roi,0,0
1,tumor,1,1
2,stroma,2,2
3,lymphocytic_infiltrate,3,3
4,necrosis_or_debris,4,4
5,glandular_secretions,5,5
6,blood,6,5
7,exclude,7,5
8,metaplasia_NOS,8,5
9,fat,9,5


In [ ]:
class_names_new = ['outside_roi','tumor','stroma','inflammatory','necrosis','other']
classes_new_int = [0,1,2,3,4,5]
grouped_classes_df = pd.DataFrame()
grouped_classes_df['class_names'] = class_names_new
grouped_classes_df['grouped_gt_values'] = classes_new_int
grouped_classes_df

,class_names,grouped_gt_values
0,outside_roi,0
1,tumor,1
2,stroma,2
3,inflammatory,3
4,necrosis,4
5,other,5


### Testing generation of single mask

In [ ]:
#testing single mask creation
testing_masks_folder = '/content/gdrive/MyDrive/BCSS_dataset/testing/new_gt_folder'

prep_dataset_obj.process_mask(mask_path='/content/gdrive/MyDrive/BCSS_dataset/masks/TCGA-A1-A0SP-DX1_xmin6798_ymin53719_MPP-0.2500.png', 
                              new_masks_folder= testing_masks_folder,
                              vfunc_to_convert = None,
                              make_3d_mask=False)

processing mask: TCGA-A1-A0SP-DX1_xmin6798_ymin53719_MPP-0.2500.png
temp path: /content/gdrive/MyDrive/BCSS_dataset/testing/new_gt_folder/TCGA-A1-A0SP-DX1_xmin6798_ymin53719_MPP-0.2500.png
finished mask: TCGA-A1-A0SP-DX1_xmin6798_ymin53719_MPP-0.2500.png


'/content/gdrive/MyDrive/BCSS_dataset/testing/new_gt_folder/TCGA-A1-A0SP-DX1_xmin6798_ymin53719_MPP-0.2500.png'

In [ ]:
#testing mask creation
# test_mask = '/content/gdrive/MyDrive/BCSS_dataset/testing/new_gt_folder/TCGA-A1-A0SP-DX1_xmin6798_ymin53719_MPP-0.2500.png'
# test = cv2.imread(test_mask,cv2.IMREAD_UNCHANGED)
# test2 = Image.open(test_mask)
# test2_array = np.asarray(test2)
# print(np.shape(test2_array))
# plt.imshow(test2,cmap='gray')

### apply function to remap pixel values for each mask

In [ ]:
#select folder to save new masks depe
new_mask_dir_2d_2class = '/content/gdrive/MyDrive/BCSS_dataset/new_masks_2d_2_class'
new_mask_dir_2d_6class = '/content/gdrive/MyDrive/BCSS_dataset/new_masks_2d_6_class'

#choose which set of masks will be generated: 2d vs 3d, 6 class vs 2 class
new_mask_folder = new_mask_dir_2d_6class

#get dimensions and n-classes from filename
dims_from_path = int(new_mask_folder.split('_')[3][0])
num_classes_from_path = int(new_mask_folder.split('_')[-2])
assert isinstance(dims_from_path, int),f"dims_from_path not an integer: {dims_from_path}"
assert isinstance(num_classes_from_path, int),"num_classes_from_path not an integer"

print(f'creating {dims_from_path}d masks with {num_classes_from_path} classes ')

if (dims_from_path == 2) and (num_classes_from_path==6):
  prep_dataset_obj.remap_ground_truth_masks(new_mask_folder,
                                            convert_to_binary=False,
                                            make_3d_mask=False)
  

creating 2d masks with 6 classes 
new masks folder already exists, confirm replace existing new masks folder (y/n)n
nothing to do, exiting


In [ ]:
#testing dimensions of masks
# heights = []
# widths = []
# channels = []

# mask_folder = '/content/gdrive/MyDrive/BCSS_dataset/new_masks_2d_6_class'
# for path in os.listdir(mask_folder):
#   abs_path = os.path.join(mask_folder,path)
#   img = cv2.imread(abs_path,cv2.IMREAD_UNCHANGED)
#   n_dims = len(np.shape(img))
#   if n_dims == 3:
#     raise ValueError(f'image {abs_path} has too many dims')

## Visualising the dataset

We need to map the different classes to a palette for visualisation of the different classes

In [ ]:
# convert dataset annotation to semantic segmentation map
palette = sns.color_palette('tab20', len(classes_new_int))                                            
palette_array = np.asarray(palette)*255
# roll array so orange is tumour 
palette_array = np.roll(palette_array, -2,axis=0)
palette_array #shows palette

array([[255., 127.,  14.],
       [255., 187., 120.],
       [ 44., 160.,  44.],
       [152., 223., 138.],
       [ 31., 119., 180.],
       [174., 199., 232.]])

In [ ]:
def visualise_seg(img_path,mask_path):
  # plt.rcParams.update({'font.size': 30})

  fig, (ax1,ax2) = plt.subplots(1,2,figsize=(30,20))
  img = cv2.imread(img_path)
  ax1.imshow(img)
  ax1.set_title('Breast Cancer Histology Slide',fontsize=30)
  ax1.axis('off')

  # fig.suptitle(f'{img_path}')

  mask_img = cv2.imread(mask_path,cv2.IMREAD_UNCHANGED)
  mask_array = np.array(mask_img)
  shape = np.shape(mask_array)

  scale_percent = 20

  new_width = int(shape[1] * scale_percent / 100)
  new_height = int(shape[0] * scale_percent / 100)

  mask_img = cv2.resize(mask_img,(new_width,new_height))
  mask_array = cv2.resize(mask_array,(new_width,new_height))
  print(f'orig shape of mask array: {shape}')
  print(f'resized shape: {new_width} x {new_height}')
  if shape[-1] == 3:
    mask_array = mask_array[:,:,0]

  #make image patches to visualise different classes
  seg_img = Image.fromarray(mask_array).convert('P')
  seg_img.putpalette(np.array(palette_array, dtype=np.uint8))
  patches = [mpatches.Patch(color=palette_array[i]/255, 
                          label=class_names_new[i]) for i in range(len(classes_new_int))]
  
  ax2.legend(handles=patches, 
             bbox_to_anchor=(1.0,0.5), #was (1.5,1)
             loc=6, 
             borderaxespad=0., 
            fontsize=25,
            title='Segmentation Classes',
            title_fontsize=25,
            fancybox=True)

  # ax2.legend(handles=patches, bbox_to_anchor=(1.5,1),loc=0, borderaxespad=0., 
  #           fontsize='large')

  ax2.set_title('Segmentation Ground Truth',fontsize=30)
  ax2.imshow(seg_img)
  ax2.axis('off')
  # print(mask_array[0])
  plt.tight_layout()
  plt.show()

for x in range(4):
  visualise_seg(img_path=prep_dataset_obj.orig_image_paths_list[x],
                mask_path=prep_dataset_obj.new_masks_paths_list[x])


## Cancer dataset augmentation

### create patches of 800 x 800 for both images and masks

In [ ]:
#testing ssplit single image and mask into patches
img = prep_dataset_obj.orig_image_paths_list[0]
mask = prep_dataset_obj.new_masks_paths_list[0]

prep_dataset_obj.split_and_save_single_image_and_mask(
                                           orig_image_path=img,
                                           mask_path=mask,
                                           output_folder='/content/gdrive/MyDrive/BCSS_dataset/testing',
                                           slice_size=800)

splitting TCGA-A1-A0SK-DX1_xmin45749_ymin25055_MPP-0 into 20 patches:
finished splitting TCGA-A1-A0SK-DX1_xmin45749_ymin25055_MPP-0


In [ ]:
patched_folder = '/content/gdrive/MyDrive/BCSS_dataset/patched_images_masks_2d_6_class'

#create image patches
prep_dataset_obj.start_splitting_all_images_masks(new_folder_for_patched=patched_folder,
                                                  mask_type='new')

it appears patch folder already exists. confirm delete existing folder (y/n)n
nothing to do, exiting


In [ ]:
prep_dataset_obj.get_patched_file_paths(patched_folder)
print(len(prep_dataset_obj.patched_image_paths_list))
print(len(prep_dataset_obj.patched_mask_paths_list))

4812
4812


In [ ]:
prep_dataset_obj.make_sure_same_images_and_masks_present()

num images not present in mask list: 0
num masks not present in image list: 0
total paths with bad filename ".png.png": 0
confirm delete unmatching images and masks (y/n)y
images and masks now all have mnatching pairs


In [ ]:
prep_dataset_obj.sanity_check(prep_dataset_obj.patched_image_paths_list,
                              prep_dataset_obj.patched_mask_paths_list)

image and mask lists are the same length
for each x image and mask should have same name
0
orig image
/content/gdrive/MyDrive/BCSS_dataset/patched_images_masks_2d_6_class/images/TCGA-A1-A0SK-DX1_xmin45749_ymin25055_MPP-0_sub_image_00.png
mask
/content/gdrive/MyDrive/BCSS_dataset/patched_images_masks_2d_6_class/masks/TCGA-A1-A0SK-DX1_xmin45749_ymin25055_MPP-0_sub_image_00.png
1
orig image
/content/gdrive/MyDrive/BCSS_dataset/patched_images_masks_2d_6_class/images/TCGA-A1-A0SK-DX1_xmin45749_ymin25055_MPP-0_sub_image_01.png
mask
/content/gdrive/MyDrive/BCSS_dataset/patched_images_masks_2d_6_class/masks/TCGA-A1-A0SK-DX1_xmin45749_ymin25055_MPP-0_sub_image_01.png
2
orig image
/content/gdrive/MyDrive/BCSS_dataset/patched_images_masks_2d_6_class/images/TCGA-A1-A0SK-DX1_xmin45749_ymin25055_MPP-0_sub_image_02.png
mask
/content/gdrive/MyDrive/BCSS_dataset/patched_images_masks_2d_6_class/masks/TCGA-A1-A0SK-DX1_xmin45749_ymin25055_MPP-0_sub_image_02.png
3
orig image
/content/gdrive/MyDrive/BCSS_da

In [ ]:
for x in range(10):
  visualise_seg(img_path=prep_dataset_obj.patched_image_paths_list[x],
                mask_path=prep_dataset_obj.patched_mask_paths_list[x])